In [14]:
"""
1. Charting
2. Performance
3. Trading Strategy
"""

import glob
from random import randint
import datetime
import openpyxl
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
from openpyxl.chart import (
LineChart,
Reference,
BarChart,
StockChart,
Series)
from openpyxl.drawing.fill import PatternFillProperties, ColorChoice
from openpyxl.chart.axis import DateAxis, ChartLines
from openpyxl.chart.updown_bars import UpDownBars
from openpyxl.drawing.image import Image

wb = Workbook()
ws = wb.active

# order of data: Date,Open,High,Low,Close,Adj Close,Volume

startDate = datetime.date(2018,9,1) #Limit the data stored on the worksheet to the period of 9/1/2018 to 6/1/2019
endDate = datetime.date(2019,6,1)
vol_sale_pct = 1.2
purchased_shares = 1000
for name in glob.glob('c:\\temp\\*_20190922.csv'):
    #print(name) 
    filePath = name.split("\\") 
    wsName = filePath[-1][:4] # separate the symbol from the path
    #print(wsName)
    with open(name,'r') as fh:
        ws = wb.create_sheet(title=wsName) # use the symbol to name the worksheets
        #print(ws)
        
        # create headers
        ws["A"+str(1)] = 'Symbol'
        ws["B"+str(1)] = 'Trade Date'
        ws["C"+str(1)] = 'Open Price'
        ws["D"+str(1)] = 'High Price'
        ws["E"+str(1)] = 'Low Price'
        ws["F"+str(1)] = 'Close Price'
        ws["G"+str(1)] = 'Adj Close'
        ws["H"+str(1)] = 'Volume'
        ws["I"+str(1)] = 'Performance'
        ws["J"+str(1)] = 'Buy/Sell'
        ws["K"+str(1)] = 'Shares Purchased'
        ws["L"+str(1)] = 'Total Purchase Price'
        
        iRow = 2 
        cnt = 0
        line = fh.readline() 
        while line:
            if cnt > 0:
                tradeData = line.split(",")
                tradeData.insert(0,wsName) # use the symbol as an id for each row
                #print(tradeData)

                tradeDate = tradeData[1]
                openPrice = tradeData[2]
                highPrice = tradeData[3]
                lowPrice = tradeData[4]
                closePrice = tradeData[5]
                adjClose = tradeData[6]
                volume = tradeData[7]

                yyyy,mm,dd = tradeDate.split("-")
                newDate = datetime.date(int(yyyy),int(mm),int(dd))
                
                # add info to excel worksheet
                if newDate >= startDate and newDate <= endDate:  
                    if iRow == 2:
                        prevPrice = closePrice
                    ws["A"+str(iRow)] = wsName
                    ws["B"+str(iRow)] = tradeDate
                    ws["C"+str(iRow)] = float(openPrice)
                    ws["D"+str(iRow)] = float(highPrice)
                    ws["E"+str(iRow)] = float(lowPrice)
                    ws["F"+str(iRow)] = float(closePrice)
                    ws["G"+str(iRow)] = float(adjClose)
                    ws["H"+str(iRow)] = float(volume)
                    ws["I"+str(iRow)] = "=($F"+str(iRow)+"/$F2)-1"
                    
                    # if the average increase in volume for the past 5 days is 20% or greater, buy 1000 shares
                    if iRow >= 7:
                        vol1 = ws["H"+str(iRow-5)].value
                        vol2 = ws["H"+str(iRow-4)].value
                        vol3 = ws["H"+str(iRow-3)].value
                        vol4 = ws["H"+str(iRow-2)].value
                        vol5 = ws["H"+str(iRow-1)].value
                        
                        avg_volume = (vol1 + vol2 + vol3 + vol4 + vol5)/5
                        current_volume = float(volume)
                        
                        if current_volume >= avg_volume * vol_sale_pct:
                            ws["J"+str(iRow)] = 'BUY'
                            ws["K"+str(iRow)] = purchased_shares
                            ws["L"+str(iRow)] = purchased_shares * float(closePrice)

                    iRow += 1
                    
            cnt += 1
            line = fh.readline()
        ws["L"+str(iRow+1)] = "=SUM($L2:L"+str(iRow)+")" # total purchase price for each buy day
        

        # Charting the security
        baseCols = 6 # 2 + len(processSecurities2)
        maxRows = iRow - 1
        # data = Reference(ws3, min_col=1, min_row=3, max_col=baseCols, max_row=maxRows)
        data = Reference(ws, min_col=6, min_row=2, max_col=6, max_row=maxRows)
        c2 = LineChart()
        # c2.title = title
        c2.style = 12
        c2.y_axis.title = "Chart security"
        c2.y_axis.crossAx = 500
        c2.x_axis = DateAxis(crossAx=100)
        #c2.x_axis.number_format = 'd-mmm'
        c2.x_axis.majorTimeUnit = "days"
        c2.x_axis.title = "Days"

        c2.add_data(data, titles_from_data=True)
        dates = Reference(ws, min_col=2, min_row=2, max_row=maxRows)
        c2.set_categories(dates)
        wsChart = wb.create_sheet(title="Chart_"+wsName)
        wsChart = wb["Chart_"+wsName]
        wsChart.add_chart(c2, "A2")  

wb.save("C:\\temp\\PERFORMANCE_ANALYSIS_20190922_" + str(randint(1,10000)) + ".xslx")